In [126]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [127]:
import galois
import numpy as np

In [128]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(2**3)

In [129]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

In [130]:
A = field.Random((k, n_k))
print(A)

[[2 0 6 1]
 [4 4 7 5]
 [2 6 4 7]]


In [131]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [132]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
print(A_primo_intermedio)

[[2 6 4 7]
 [2 0 6 1]
 [4 4 7 5]]


In [133]:
A_primo = field(np.dot(A_primo_intermedio, matrice_bin2))
print(A_primo)

[[2 7 6 4]
 [2 1 0 6]
 [4 5 4 7]]


In [134]:
import threading

In [135]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

#creo il lock per evitare conglitti nell'aggiornamento
lock = threading.Lock()

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%8)

    print("Riga : ", riga) #questi vanno eliminati poi
    print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    print("Somma: ", somma) #questi vanno eliminati poi
    print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    with lock: #vado ad usare il lock
        somme_rig[indice_riga] = somma
        print(somme_rig)

def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    with lock:
        primi_colonne[indice_colonna] = colonna[0]
        print(primi_colonne)
        secondi_colonne[indice_colonna] = colonna[1]
        print(secondi_colonne)


In [136]:
threads_righe = []
threds_colonne = []
colonne_finali = []

In [137]:
#creo un thread per ogni colonna
for indice_riga in range(A_primo.shape[0]):
    t = threading.Thread(target = somma_riga, args = (A_primo, indice_riga))
    threads_righe.append(t)
    t.start() #avvio il thread

#aspetto il risultato di tutti i threads
for t in threads_righe:
    t.join()


Riga :  [2 7 6 4]
Tipo riga:  <class 'galois.GF(2^3)'>
Somma:  3
Tipo somma:  <class 'galois.GF(2^3)'>
[GF(3, order=2^3), GF(0, order=2^3), GF(0, order=2^3)]
Riga :  [2 1 0 6]
Tipo riga:  <class 'galois.GF(2^3)'>
Somma:  1
Tipo somma:  <class 'galois.GF(2^3)'>
[GF(3, order=2^3), GF(1, order=2^3), GF(0, order=2^3)]
Riga :  [4 5 4 7]
Tipo riga:  <class 'galois.GF(2^3)'>
Somma:  4
Tipo somma:  <class 'galois.GF(2^3)'>
[GF(3, order=2^3), GF(1, order=2^3), GF(4, order=2^3)]


In [138]:
print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)
print ("\nElementi primi estratti: ", primi_colonne)
print ("\nElementi secondi estratti: ", secondi_colonne)


Somma degli elementi di ciascuna riga:  [GF(3, order=2^3), GF(1, order=2^3), GF(4, order=2^3)]

Elementi primi estratti:  [0, 0, 0, 0]

Elementi secondi estratti:  [0, 0, 0, 0]


In [139]:
#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
print(indici_ordinati)

#applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]

print(A_canonica_righe)

[1 0 2]
[[2 1 0 6]
 [2 7 6 4]
 [4 5 4 7]]


In [140]:
#ora si passa alle colonne
for indice_colonna in range(A_primo.shape[1]):
    p = threading.Thread(target = estrai_da_colonna, args = (A_canonica_righe, indice_colonna))
    threds_colonne.append(p)
    p.start()

for p in threds_colonne:
    p.join()

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], idx))
    print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1]))

A_CF = A_canonica_righe[:, [col[2] for col in colonne_finali]]

print(A_CF)

[GF(2, order=2^3), 0, 0, 0]
[GF(2, order=2^3), 0, 0, 0]
[GF(2, order=2^3), GF(1, order=2^3), 0, 0]
[GF(2, order=2^3), GF(7, order=2^3), 0, 0]
[GF(2, order=2^3), GF(1, order=2^3), GF(0, order=2^3), 0]
[GF(2, order=2^3), GF(7, order=2^3), GF(6, order=2^3), 0]
[GF(2, order=2^3), GF(1, order=2^3), GF(0, order=2^3), GF(6, order=2^3)]
[GF(2, order=2^3), GF(7, order=2^3), GF(6, order=2^3), GF(4, order=2^3)]
Index:  2
Colonne finali:  [(GF(0, order=2^3), GF(6, order=2^3), np.int64(2))]
Index:  1
Colonne finali:  [(GF(0, order=2^3), GF(6, order=2^3), np.int64(2)), (GF(1, order=2^3), GF(7, order=2^3), np.int64(1))]
Index:  0
Colonne finali:  [(GF(0, order=2^3), GF(6, order=2^3), np.int64(2)), (GF(1, order=2^3), GF(7, order=2^3), np.int64(1)), (GF(2, order=2^3), GF(2, order=2^3), np.int64(0))]
Index:  3
Colonne finali:  [(GF(0, order=2^3), GF(6, order=2^3), np.int64(2)), (GF(1, order=2^3), GF(7, order=2^3), np.int64(1)), (GF(2, order=2^3), GF(2, order=2^3), np.int64(0)), (GF(6, order=2^3), GF(4, 

In [141]:
#devo fare lo stesso per la matrice per dimostrare che sono CF

threads_righe = []
threds_colonne = []
colonne_finali = []

#creo un thread per ogni colonna
for indice_riga in range(A_primo.shape[0]):
    t = threading.Thread(target = somma_riga, args = (A, indice_riga))
    threads_righe.append(t)
    t.start() #avvio il thread

#aspetto il risultato di tutti i threads
for t in threads_righe:
    t.join()

print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

print(A_canonica_righe)

#ora si passa alle colonne
for indice_colonna in range(A_primo.shape[1]):
    p = threading.Thread(target = estrai_da_colonna, args = (A_canonica_righe, indice_colonna))
    threds_colonne.append(p)
    p.start()

for p in threds_colonne:
    p.join()

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], idx))
    print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1]))

A_CF = A_canonica_righe[:, [col[2] for col in colonne_finali]]

print(A_CF)

Riga :  [2 0 6 1]
Tipo riga:  <class 'galois.GF(2^3)'>
Somma:  1
Tipo somma:  <class 'galois.GF(2^3)'>
[GF(1, order=2^3), GF(1, order=2^3), GF(4, order=2^3)]
Riga :  [4 4 7 5]
Tipo riga:  <class 'galois.GF(2^3)'>
Somma:  4
Tipo somma:  <class 'galois.GF(2^3)'>
[GF(1, order=2^3), GF(4, order=2^3), GF(4, order=2^3)]
Riga :  [2 6 4 7]
Tipo riga:  <class 'galois.GF(2^3)'>
Somma:  3
Tipo somma:  <class 'galois.GF(2^3)'>
[GF(1, order=2^3), GF(4, order=2^3), GF(3, order=2^3)]

Somma degli elementi di ciascuna riga:  [GF(1, order=2^3), GF(4, order=2^3), GF(3, order=2^3)]
[0 2 1]
[[2 0 6 1]
 [2 6 4 7]
 [4 4 7 5]]
[GF(2, order=2^3), GF(1, order=2^3), GF(0, order=2^3), GF(6, order=2^3)]
[GF(2, order=2^3), GF(7, order=2^3), GF(6, order=2^3), GF(4, order=2^3)]
[GF(2, order=2^3), GF(0, order=2^3), GF(0, order=2^3), GF(6, order=2^3)]
[GF(2, order=2^3), GF(6, order=2^3), GF(6, order=2^3), GF(4, order=2^3)]
[GF(2, order=2^3), GF(0, order=2^3), GF(6, order=2^3), GF(6, order=2^3)]
[GF(2, order=2^3), GF(6